In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [1]:
def load_data(data_dir):
    X = []
    y = []
    classes = 43
    for i in range(classes):
        path = os.path.join(data_dir, str(i))
        print(f"Loading class {i}...")  # 🟨 progress track karne ke liye
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path, img))
                image = cv2.resize(image, (32, 32))
                X.append(image)
                y.append(i)
            except Exception as e:
                pass
    return np.array(X), np.array(y)


In [5]:
data_dir = "C:/Users/LHCP/Desktop/road_signal_detection/dataset/Train"
X, y = load_data(data_dir)

print("✅ Dataset loaded successfully!")
print("X shape:", X.shape)
print("y shape:", y.shape)


Loading class 0...
Loading class 1...
Loading class 2...
Loading class 3...
Loading class 4...
Loading class 5...
Loading class 6...
Loading class 7...
Loading class 8...
Loading class 9...
Loading class 10...
Loading class 11...
Loading class 12...
Loading class 13...
Loading class 14...
Loading class 15...
Loading class 16...
Loading class 17...
Loading class 18...
Loading class 19...
Loading class 20...
Loading class 21...
Loading class 22...
Loading class 23...
Loading class 24...
Loading class 25...
Loading class 26...
Loading class 27...
Loading class 28...
Loading class 29...
Loading class 30...
Loading class 31...
Loading class 32...
Loading class 33...
Loading class 34...
Loading class 35...
Loading class 36...
Loading class 37...
Loading class 38...
Loading class 39...
Loading class 40...
Loading class 41...
Loading class 42...
✅ Dataset loaded successfully!
X shape: (39209, 32, 32, 3)
y shape: (39209,)


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Step 1: Normalize pixel values
X = X / 255.0

# Step 2: One-hot encode labels
y = to_categorical(y, num_classes=43)

# Step 3: Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Data prepared successfully!")
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)


✅ Data prepared successfully!
Training data shape: (31367, 32, 32, 3)
Validation data shape: (7842, 32, 32, 3)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step: Create CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # to reduce overfitting
model.add(Dense(43, activation='softmax'))  # 43 classes

# Step: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("✅ CNN model created and compiled!")


C:\Users\LHCP\anaconda3\envs\road-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ CNN model created and compiled!


In [8]:
# Step: Train the CNN model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, y_val),
    verbose=1
)


Epoch 1/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.4707 - loss: 1.9189 - val_accuracy: 0.8623 - val_loss: 0.5971
Epoch 2/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step - accuracy: 0.7987 - loss: 0.6510 - val_accuracy: 0.9566 - val_loss: 0.2262
Epoch 3/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step - accuracy: 0.8757 - loss: 0.4016 - val_accuracy: 0.9675 - val_loss: 0.1365
Epoch 4/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9079 - loss: 0.2961 - val_accuracy: 0.9712 - val_loss: 0.1051
Epoch 5/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9258 - loss: 0.2389 - val_accuracy: 0.9824 - val_loss: 0.0819
Epoch 6/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9393 - loss: 0.1948 - val_accuracy: 0.9869 - val_loss: 0.0611
Epoch 7/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9488 - loss: 0.1627 - val_accuracy: 0.9897 - val_loss: 0.0438
Epoch 8/10
491/491 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9547 - loss: 0.1458 - 

In [9]:
# Step: Evaluate the trained model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print("✅ Validation Accuracy: {:.2f}%".format(val_accuracy * 100))


246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9911 - loss: 0.0352
✅ Validation Accuracy: 99.11%


In [10]:
# Step: Save the trained model
model.save("road_sign_cnn_model.h5")
print("✅ Model saved successfully!")


✅ Model saved successfully!
